In [23]:
%pip install onnxruntime-gpu --extra-index-url https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/
%pip install fastembed-gpu qdrant-client[fastembed-gpu] elasticsearch tqdm unstructured ipywidgets jsonlines pandas pyarrow numpy transformers torch onnx dateparser spacy[cuda12x] nvidia-ml-py3

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 MB 14.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install nvidia-ml-py3

  Using cached nvidia_ml_py3-7.352.0-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [5]:
!python -m spacy download nl_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.1/568.1 MB 14.3 MB/s eta 0:00:0000:0100:02
  Using cached spacy-3.7.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (27 kB)
Using cached spacy-3.7.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (29.0 MB)
  Attempting uninstall: spacy
    Found existing installation: spacy 3.8.0
    Uninstalling spacy-3.8.0:
      Successfully uninstalled spacy-3.8.0
✔ Download and installation successful
You can now load the package via spacy.load('nl_core_news_lg')


In [40]:
from transformers import BertTokenizer, BertForTokenClassification
from transformers.onnx import OnnxConfig
import torch
from transformers.onnx import export
from pathlib import Path

class TokenClassificationOnnxConfig(OnnxConfig):
    def __init__(self, model_config):
        self.model_config = model_config

    @property
    def inputs(self):
        return {
            "input_ids": {0: "batch_size", 1: "sequence_length"},
            "attention_mask": {0: "batch_size", 1: "sequence_length"},
            "token_type_ids": {0: "batch_size", 1: "sequence_length"}
        }

    @property
    def outputs(self):
        return self._tasks_to_common_outputs["token-classification"]

def export_onnx_model():
    # Load the pre-trained Hugging Face model and tokenizer
    model_name = "Davlan/bert-base-multilingual-cased-ner-hrl"
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertForTokenClassification.from_pretrained(model_name)
    config = TokenClassificationOnnxConfig(model.config)

    # Define input names and shapes for the ONNX conversion
    output_path = Path("bert_base_multilingual_cased_ner_hrl.onnx")
    # dummy_input = tokenizer("Hoi, Bron is een tool van de Open State Foundation", return_tensors="pt")

    # Export the model to ONNX
    export(
        model=model,
        preprocessor=tokenizer,
        output=output_path,
        opset=13,  # ONNX opset version
        config=config,
        # input_shapes={"input_ids": dummy_input["input_ids"].shape},
        device="cuda" if torch.cuda.is_available() else "cpu",
    )

    print(f"ONNX model exported to {output_path}")

export_onnx_model()

AttributeError: 'TokenClassificationOnnxConfig' object has no attribute '_config'

In [11]:
import sys
import re
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
import pandas as pd
from tqdm import tqdm
import multiprocessing
from multiprocessing import Pool
import pyarrow as pa
import pyarrow.parquet as pq
from unstructured.partition.html import partition_html
from unstructured.cleaners.core import (
    clean,
    clean_non_ascii_chars,
    replace_unicode_quotes,
    group_broken_paragraphs
)
from lxml import etree

def remove_processing_instructions(html_text):
    """Remove processing instructions from HTML content using lxml."""
    try:
        parser = etree.HTMLParser(remove_pis=True)
        tree = etree.fromstring(html_text.encode('utf-8'), parser)
        return etree.tostring(tree, encoding='unicode', method='html')
    except Exception as e:
        # If parsing fails, return the original text
        tqdm.write(f"Error removing processing instructions: {e}", file=sys.stderr)
        return html_text

def process_document(doc):
    try:
        # Extract the 'description' field
        description = doc.get('_source', {}).get('description', '')
        description = remove_processing_instructions(description)

        # Process 'description' field using unstructured.io
        try:
            elements = partition_html(
                text=description, 
                chunking_strategy='by_title', 
                combine_text_under_n_chars=1024,
                max_characters=4096,
                new_after_n_chars=3072,
                overlap=256
            )
        except Exception as e:
            tqdm.write(f"Exception in partition_html for doc {doc.get('_id')}: {e}", file=sys.stderr)
            return None

        docs = []
        chunk_count = len(elements)

        for i, element in enumerate(elements):
            try:
                # Skip elements that are processing instructions
                if hasattr(element, 'element') and isinstance(element.element, etree._ProcessingInstruction):
                    continue

                if hasattr(element, 'text') and element.text:
                    cleaned_text = clean(
                        element.text,
                        extra_whitespace=True,
                        dashes=True,
                        bullets=True,
                        trailing_punctuation=False,
                        lowercase=False
                    )
                    cleaned_text = clean_non_ascii_chars(cleaned_text)
                    cleaned_text = replace_unicode_quotes(cleaned_text)
                    cleaned_text = group_broken_paragraphs(cleaned_text)

                    docs.append({
                        "text": cleaned_text,
                        "es_id": doc["_id"],
                        "title": doc["_source"].get("title", ""),
                        "location": doc["_source"].get("location", ""),
                        "location_name": doc["_source"].get("location_name", ""),
                        "modified": doc["_source"].get("modified", ""),
                        "published": doc["_source"].get("published", ""),
                        "source": doc["_source"].get("source", ""),
                        "type": doc["_source"].get("type", ""),
                        "identifier": doc["_source"].get("identifier", ""),
                        "url": doc["_source"].get("url", ""),
                        "chunk_index": i,
                        "chunk_count": chunk_count
                    })
                else:
                    # Skip elements without text
                    continue
            except Exception as e:
                tqdm.write(f"Exception processing element in doc {doc.get('_id')}: {e}", file=sys.stderr)
                continue

        return docs if docs else None
    except Exception as e:
        # Log exception and return None
        tqdm.write(f"Exception processing doc {doc.get('_id')}: {e}", file=sys.stderr)
        return None

def main():
    # Connect to Elasticsearch
    es = Elasticsearch("http://localhost:9200")
    index_name = "jodal_documents7"
    parquet_file_3_gemeentes = '3_gemeentes.parquet'
    parquet_file_overijssel = 'overijssel.parquet'
    parquet_file_all = 'all.parquet'
    query_3_gemeentes = {
        "query": {
            "bool": {
                "must": [
                    {"exists": {"field": "description"}}
                ],
                "minimum_should_match": 1,
                "should": [
                    {
                        "match_phrase": {
                        "location": "GM0141"
                        }
                    },
                    {
                        "match_phrase": {
                        "location": "GM1896"
                        }
                    },
                    {
                        "match_phrase": {
                        "location": "GM0180"
                        }
                    }
                ]
            }
        }
    }
    query_overijssel = {
        "query": {
            "bool": {
                "must": [
                    {"exists": {"field": "description"}}
                ],
                "minimum_should_match": 1,
                "should": [
                    {
                        "match_phrase": {
                        "location": "GM0193"
                        }
                    },
                    {
                        "match_phrase": {
                        "location": "GM0141"
                        }
                    },
                    {
                        "match_phrase": {
                        "location": "GM0166"
                        }
                    },
                    {
                        "match_phrase": {
                        "location": "GM1774"
                        }
                    },
                    {
                        "match_phrase": {
                        "location": "GM0183"
                        }
                    },
                    {
                        "match_phrase": {
                        "location": "GM0173"
                        }
                    },
                    {
                        "match_phrase": {
                        "location": "GM0150"
                        }
                    },
                    {
                        "match_phrase": {
                        "location": "GM1700"
                        }
                    },
                    {
                        "match_phrase": {
                        "location": "GM0164"
                        }
                    },
                    {
                        "match_phrase": {
                        "location": "GM0153"
                        }
                    },
                    {
                        "match_phrase": {
                        "location": "GM0148"
                        }
                    },
                    {
                        "match_phrase": {
                        "location": "GM1708"
                        }
                    },
                    {
                        "match_phrase": {
                        "location": "GM0168"
                        }
                    },
                    {
                        "match_phrase": {
                        "location": "GM0160"
                        }
                    },
                    {
                        "match_phrase": {
                        "location": "GM0189"
                        }
                    },
                    {
                        "match_phrase": {
                        "location": "GM0177"
                        }
                    },
                    {
                        "match_phrase": {
                        "location": "GM1742"
                        }
                    },
                    {
                        "match_phrase": {
                        "location": "GM0180"
                        }
                    },
                    {
                        "match_phrase": {
                        "location": "GM1896"
                        }
                    },
                    {
                        "match_phrase": {
                        "location": "GM0175"
                        }
                    },
                    {
                        "match_phrase": {
                        "location": "GM1735"
                        }
                    },
                    {
                        "match_phrase": {
                        "location": "GM0147"
                        }
                    },
                    {
                        "match_phrase": {
                        "location": "GM0163"
                        }
                    },
                    {
                        "match_phrase": {
                        "location": "GM0158"
                        }
                    },
                    {
                        "match_phrase": {
                        "location": "GM1773"
                        }
                    }
                ]
            }
        }
    }

    query_all = {
        "query": {
            "bool": {
                "must": [
                    {"exists": {"field": "description"}}
                ]
            }
        }
    }

    query = query_overijssel
    parquet_file = parquet_file_overijssel

    # Use scan to retrieve documents
    es_scan = scan(client=es, index=index_name, query=query)

    # Get total document count for progress bar
    total_docs = es.count(index=index_name, body=query)['count']

    max_workers = multiprocessing.cpu_count()
    pool = Pool(processes=max_workers)
    processed_docs_iter = pool.imap_unordered(process_document, es_scan, chunksize=100)

    processed_docs = []
    batch_size = 1000  # Adjust based on your memory capacity
    batch_num = 0

    # Use a unique directory for batch Parquet files
    output_dir = 'output_parquet_batches'
    import os
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with tqdm(total=total_docs) as pbar:
        for docs_list in processed_docs_iter:
            # Update the progress bar for each Elasticsearch document processed
            pbar.update(1)
            if docs_list:
                processed_docs.extend(docs_list)
                if len(processed_docs) >= batch_size:
                    df = pd.DataFrame(processed_docs)
                    # Remove columns that are problematic for Parquet
                    for col in df.columns:
                        if df[col].apply(lambda x: isinstance(x, dict) and not x).all():
                            df.drop(columns=[col], inplace=True)
                    # Write each batch to a separate Parquet file
                    batch_file = os.path.join(output_dir, f'output_part{batch_num}.parquet')
                    df.to_parquet(batch_file, engine='pyarrow', index=False)
                    processed_docs = []
                    batch_num += 1
    # Write any remaining documents
    if processed_docs:
        df = pd.DataFrame(processed_docs)
        # Remove columns that are problematic for Parquet
        for col in df.columns:
            if df[col].apply(lambda x: isinstance(x, dict) and not x).all():
                df.drop(columns=[col], inplace=True)
        batch_file = os.path.join(output_dir, f'output_part{batch_num}.parquet')
        df.to_parquet(batch_file, engine='pyarrow', index=False)
        processed_docs = []

    pool.close()
    pool.join()

    # Combine all the Parquet files into a single dataset
    import glob
    all_files = glob.glob(os.path.join(output_dir, 'output_part*.parquet'))
    tables = [pq.read_table(f) for f in all_files]
    # Update the following line to address the FutureWarning
    combined_table = pa.concat_tables(tables, promote_options='default')
    pq.write_table(combined_table, parquet_file)
    
    # Optionally, clean up batch files
    for f in all_files:
        os.remove(f)
    os.rmdir(output_dir)

main()


  0%|          | 0/247171 [00:00<?, ?it/s]Error removing processing instructions: Type 'NoneType' cannot be serialized.
Error removing processing instructions: Type 'NoneType' cannot be serialized.
Error removing processing instructions: Type 'NoneType' cannot be serialized.
Error removing processing instructions: Type 'NoneType' cannot be serialized.
Error removing processing instructions: Type 'NoneType' cannot be serialized.
Error removing processing instructions: Type 'NoneType' cannot be serialized.
Error removing processing instructions: Type 'NoneType' cannot be serialized.
Error removing processing instructions: Type 'NoneType' cannot be serialized.
Error removing processing instructions: Type 'NoneType' cannot be serialized.
Error removing processing instructions: Type 'NoneType' cannot be serialized.
Error removing processing instructions: Type 'NoneType' cannot be serialized.
Error removing processing instructions: Type 'NoneType' cannot be serialized.
Error removing process

In [1]:
import spacy
spacy.require_gpu()

True

In [2]:
import jsonlines
import time
from typing import List, Tuple
from qdrant_client import QdrantClient
from fastembed.late_interaction import LateInteractionTextEmbedding
from fastembed.sparse import SparseTextEmbedding, SparseEmbedding
from fastembed.text import TextEmbedding
import pandas as pd

from qdrant_client.models import (
    Distance,
    NamedSparseVector,
    NamedVector,
    SparseVector,
    PointStruct,
    SearchRequest,
    SparseIndexParams,
    SparseVectorParams,
    VectorParams,
    ScoredPoint,
    MultiVectorConfig,
    MultiVectorComparator
)
import concurrent.futures
import onnxruntime as ort

from transformers import BertTokenizer, BertForTokenClassification
from transformers import pipeline
import torch
import spacy
from tqdm import tqdm
import dateparser

NUM_WORKERS = 16

# Initialize Qdrant client
qdrant_client = QdrantClient(
    host="localhost",
    port=6333
)

# Create two separate inference sessions, one for each GPU
session_options_0 = ort.SessionOptions()
session_options_0.execution_mode = ort.ExecutionMode.ORT_PARALLEL
session_options_0.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL
session_options_0.intra_op_num_threads = NUM_WORKERS

session_options_1 = ort.SessionOptions()
session_options_1.execution_mode = ort.ExecutionMode.ORT_PARALLEL
session_options_1.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL
session_options_1.intra_op_num_threads = NUM_WORKERS

session_options_2 = ort.SessionOptions()
session_options_2.execution_mode = ort.ExecutionMode.ORT_PARALLEL
session_options_2.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL
session_options_2.intra_op_num_threads = NUM_WORKERS

session_options_3 = ort.SessionOptions()
session_options_3.execution_mode = ort.ExecutionMode.ORT_PARALLEL
session_options_3.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL
session_options_3.intra_op_num_threads = NUM_WORKERS


# Initialize the models on different GPUs
dense_document_embedder_0 = TextEmbedding(
    model_name="intfloat/multilingual-e5-large",
    session_options=session_options_0,
    providers=[("CUDAExecutionProvider", {"device_id": 0})]
)

dense_document_embedder_1 = TextEmbedding(
    model_name="intfloat/multilingual-e5-large",
    session_options=session_options_1,
    providers=[("CUDAExecutionProvider", {"device_id": 1})]
)

sparse_document_embedder_0 = SparseTextEmbedding(
    model_name="Qdrant/bm25",
    session_options=session_options_2,
    providers=[("CUDAExecutionProvider", {"device_id": 0})]
)

sparse_document_embedder_1 = SparseTextEmbedding(
    model_name="Qdrant/bm25",
    session_options=session_options_3,
    providers=[("CUDAExecutionProvider", {"device_id": 1})]
)

# late_interaction_document_embedder = LateInteractionTextEmbedding(
#     model_name="colbert-ir/colbertv2.0",
#     session_options=session_options_2,
#     providers=[("CUDAExecutionProvider", {"device_id": 1})]    
# )

# Load the small Dutch model

# def make_late_interaction_embedding(texts: List[str]):
#     return list(late_interaction_document_embedder.embed(texts))

# def make_sparse_embedding(texts: List[str]):
#     return list(sparse_document_embedder.embed(texts))

# def embed_documents_on_gpu(embedder, documents, progress_bar):
#     embeddings = []
#     for doc in documents:
#         embedding = embedder.embed(doc)  # Replace with actual embedding generation logic
#         embeddings.append(embedding)
#         progress_bar.update(1)
#     return embeddings

def embed_documents_on_gpu(embedder, documents, progress_bar):
    # 'embedder.embed(documents)' returns a generator over embeddings
    embedding_gen = embedder.embed(documents)
    embeddings = []
    for embedding in embedding_gen:
        embeddings.append(embedding)
        progress_bar.update(1)
    return embeddings

def make_sparse_embedding(texts: List[str]) -> List[SparseEmbedding]:
    total_docs = len(texts)
    
    # Split documents between the two GPUs
    mid_point = total_docs // 2
    docs_0 = texts[:mid_point]
    docs_1 = texts[mid_point:]

    with tqdm(total=total_docs, desc="Generating sparse embeddings") as progress_bar:
        # Run inference on both GPUs in parallel
        with concurrent.futures.ThreadPoolExecutor() as executor:
            future_0 = executor.submit(embed_documents_on_gpu, sparse_document_embedder_0, docs_0, progress_bar)
            future_1 = executor.submit(embed_documents_on_gpu, sparse_document_embedder_1, docs_1, progress_bar)
            
            embeddings_0 = future_0.result()
            embeddings_1 = future_1.result()

    embeddings = embeddings_0 + embeddings_1
    return embeddings

# Function to embed documents on a specific embedder
# def embed_documents_on_gpu(embedder, documents):
#     return list(embedder.embed(documents))

def make_dense_embedding(texts: List[str]):    
    # Split documents between the two GPUs
    mid_point = len(texts) // 2
    docs_0 = texts[:mid_point]
    docs_1 = texts[mid_point:]

    # Run inference on both GPUs in parallel
    total_docs = len(texts)
    with tqdm(total=total_docs, desc="Generating dense embeddings") as progress_bar:
        with concurrent.futures.ThreadPoolExecutor() as executor:
            future_0 = executor.submit(embed_documents_on_gpu, dense_document_embedder_0, docs_0, progress_bar)
            future_1 = executor.submit(embed_documents_on_gpu, dense_document_embedder_1, docs_1, progress_bar)
            
            embeddings_0 = future_0.result()
            embeddings_1 = future_1.result()

    # Combine results
    embeddings = embeddings_0 + embeddings_1
    return embeddings

def parse_date(input_string):
    date = dateparser.parse(input_string, languages=['nl'])
    if date is None:
        return None
    else:
        return date.strftime('%Y-%m-%d')
    
def make_qdrant_points(df: pd.DataFrame) -> List[PointStruct]:
    print("Creating Qdrant points")
    sparse_vectors = df["sparse_embedding"].tolist()
    combined_text = df["combined_text"].tolist()
    dense_vectors = df["dense_embedding"].tolist()
    # late_interaction_vector = df["late_interaction"].tolist()
    rows = df.to_dict(orient="records")
    points = []
    for idx, (combined_text, sparse_vector, dense_vector) in enumerate(
        zip(combined_text, sparse_vectors, dense_vectors)
    ):
        sparse_vector = SparseVector(
            indices=sparse_vector.indices.tolist(), 
            values=sparse_vector.values.tolist()
        )
        
        # prepare the entities for the qdrant points
        entities, dates, locations, people, organisations, times = [], [], [], [], [], []
        entity_types = ["DATE", "GPE", "PERSON", "ORG", "TIME"]
        for entity in rows[idx]["entities"].ents:
            if entity.label_ in entity_types:
                entities.append({
                    "text": entity.text,  # Use dot notation to access attributes
                    "label": entity.label_,
                    "start": entity.start_char,
                    "end": entity.end_char,
                })

                if entity.label_ == "DATE":
                    if entity.text not in dates:
                        date = parse_date(entity.text)
                        if date is not None:
                            dates.append(date)
                elif entity.label_ == "GPE":
                    if entity.text not in locations:
                        locations.append(entity.text)                
                elif entity.label_ == "PERSON":
                    if entity.text not in people:
                        people.append(entity.text)
                elif entity.label_ == "ORG":
                    if entity.text not in organisations:
                        organisations.append(entity.text)

        point = PointStruct(
            id=idx,
            payload={
                "title": rows[idx]["title"],
                "location_name": rows[idx]["location_name"],
                "location": rows[idx]["location"],
                "text": combined_text,
                "source": rows[idx]["source"],
                "es_id": rows[idx]["es_id"],
                "modified": rows[idx]["modified"],
                "published": rows[idx]["published"],
                "type": rows[idx]["type"],
                "identifier": rows[idx]["identifier"],
                "url": rows[idx]["url"],
                "ner_entities": entities,
                "ner_dates": dates,
                "ner_locations": locations,
                "ner_people": people,
                "ner_organisations": organisations,
                "ner_times": times,
                "chunk_index": rows[idx]["chunk_index"],
                "chunk_count": rows[idx]["chunk_count"],
            },  # Add any additional payload if necessary
            vector={
                "text-sparse": sparse_vector,
                # "text-late-interaction": late_interaction_vector,      
                "text-dense": dense_vector.tolist(),          
            },
        )
        points.append(point)

    return points

def ner_pipeline(docs, gpu_id, progress_bar):
    entities = []
    torch.cuda.set_device(gpu_id)
    spacy.require_gpu()
    nlp = spacy.load("nl_core_news_lg")
    for doc in nlp.pipe(docs, batch_size=100):
        entities.append(doc)
        progress_bar.update(1)
    return entities

def run_ner_pipeline(texts: List[str]):
    print( "Running NER pipeline")
    # Split documents between the two GPUs
    mid_point = len(texts) // 2
    docs_0 = texts[:mid_point]
    docs_1 = texts[mid_point:]

    # Run inference on both GPUs in parallel
    total_docs = len(texts)
    with tqdm(total=total_docs, desc="Running NER pipeline") as progress_bar:
        with concurrent.futures.ThreadPoolExecutor() as executor:
            future_0 = executor.submit(ner_pipeline, docs_0, 0, progress_bar)
            future_1 = executor.submit(ner_pipeline, docs_1, 1, progress_bar)
            
            entities_0 = future_0.result()
            entities_1 = future_1.result()

    # Combine results
    entities = entities_0 + entities_1
    return entities
    

def upsert_with_progress(collection_name, points, batch_size=1000):
    qdrant_client.create_collection(
        collection_name,
        vectors_config={
            "text-dense": VectorParams(
                size=1024,  # OpenAI Embeddings
                distance=Distance.COSINE,
            ),
            # "text-late-interaction": VectorParams(
            #     size=5, 
            #     distance=Distance.COSINE,
            #     multivector_config=MultiVectorConfig(
            #         comparator=MultiVectorComparator.MAX_SIM
            #     ),
            # ),
        },
        sparse_vectors_config={
            "text-sparse": SparseVectorParams(
                index=SparseIndexParams(
                    on_disk=False,
                )
            )
        },
    )

    total_points = len(points)
    with tqdm(total=total_points, desc="Upserting points") as progress_bar:
        for i in range(0, total_points, batch_size):
            batch = points[i:i + batch_size]
            qdrant_client.upsert(collection_name, batch)
            progress_bar.update(len(batch))

# load the data from parquet
def load_data(df, collection_name):
    # increment collection name based on existing collections
    from qdrant_client.models import CollectionsResponse
    collections : CollectionsResponse = qdrant_client.get_collections()
    collection_name = f"{collection_name}-{len(collections.collections)}"    
    
    # create a new field which combines the fields text, title and location_name into into a single text  like this f'Titel: {value}. Gemeente:  {value}. Tekst: {value}'
    df["combined_text"] = df.apply(lambda x: f'Titel: {x["title"]} \n Gemeente: {x["location_name"]} \n Tekst: \n\n {x["text"]}', axis=1)

    combined_texts = df["combined_text"].tolist()

    # df["dense_embedding"] = make_dense_embedding(combined_texts)
    # df["sparse_embedding"] = make_sparse_embedding(combined_texts)
    df["entities"] = run_ner_pipeline(combined_texts)

    points = make_qdrant_points(df)

    upsert_with_progress(collection_name, points)

    return df

def run(collection_name, parquet_file, docs_to_load = None):
    df = pd.read_parquet(parquet_file, engine="pyarrow")

    if docs_to_load is not None:
        df = df[:docs_to_load]

    df = load_data(df, collection_name)

    return df



Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

2024-09-13 11:30:49.743341346 [W:onnxruntime:, session_state.cc:1166 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2024-09-13 11:30:49.743353128 [W:onnxruntime:, session_state.cc:1168 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

2024-09-13 11:30:51.057085792 [W:onnxruntime:, session_state.cc:1166 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2024-09-13 11:30:51.057098496 [W:onnxruntime:, session_state.cc:1168 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.


Fetching 29 files:   0%|          | 0/29 [00:00<?, ?it/s]

Fetching 29 files:   0%|          | 0/29 [00:00<?, ?it/s]

In [ ]:
run("3-Gemeentes", "3_gemeentes.parquet")

Running NER pipeline


Running NER pipeline:   7%|▋         | 44501/629672 [01:12<12:30, 779.95it/s]

KeyboardInterrupt: 

In [ ]:
from transformers import BertTokenizer, BertForTokenClassification
from transformers import pipeline
import torch

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained("Davlan/bert-base-multilingual-cased-ner-hrl")
model = BertForTokenClassification.from_pretrained("Davlan/bert-base-multilingual-cased-ner-hrl")

# Create a pipeline for NER
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)

# Sample text
text = "Albert Einstein was born in Ulm, Germany."

# Perform NER
entities = ner_pipeline(text)
print(entities)

In [29]:
import numpy as np

def search(query_text: str):
    # # Compute sparse and dense vectors
    query_sparse_vectors: List[SparseEmbedding] = make_sparse_embedding([query_text])
    query_dense_vector: List[np.ndarray] = make_dense_embedding([query_text])

    search_results = qdrant_client.search_batch(
        collection_name=collection_name,
        requests=[
            SearchRequest(
                vector=NamedVector(
                    name="text-dense",
                    vector=query_dense_vector[0].tolist(),
                ),
                limit=10,
                with_payload=True,
            ),
            SearchRequest(
                vector=NamedSparseVector(
                    name="text-sparse",
                    vector=SparseVector(
                        indices=query_sparse_vectors[0].indices.tolist(),
                        values=query_sparse_vectors[0].values.tolist(),
                    ),
                ),
                limit=10,
                with_payload=True,
            ),
        ],
    )

    return search_results

query = "Welke toezegging doet wethouder Mussche?"
search_results = search(query)
print(search_results)

[[ScoredPoint(id=3, version=0, score=0.85976475, payload={'title': 'Conceptbesluitenlijst raadsvergadering 1feb2022.docx', 'location_name': 'Staphorst', 'location': 'GM0180', 'text': 'Titel: Conceptbesluitenlijst raadsvergadering 1feb2022.docx \n Gemeente: Staphorst \n Tekst: \n\n warmte, en ingekomen stuk 1213: NLVOW: oproep aan gemeentes tot gebruik van gedragscodes. Wethouder Mussche geeft antwoord op de gestelde vragen. b. Collegeberichten Er zijn geen collegeberichten.', 'source': 'openbesluitvorming', 'es_id': '651b1e224f177a6bf2bca6916f7fda7368762716', 'modified': '2022-02-15T19:30:00+00:00', 'published': '2022-02-15T19:30:00+00:00', 'type': 'Bestand', 'identifier': 'https://id.openraadsinformatie.nl/4318661', 'url': 'https://openbesluitvorming.nl/?zoekterm=%22*%22&organisaties=%5B%22ori_staphorst_20190809125107%22%5D&showResource=https%253A%252F%252Fid.openraadsinformatie.nl%252F4318661', 'chunk_index': 3, 'chunk_count': 14}, vector=None, shard_key=None, order_value=None), Scor

In [31]:
def rrf(rank_lists, alpha=60, default_rank=1000):
    """
    Optimized Reciprocal Rank Fusion (RRF) using NumPy for large rank lists.

    :param rank_lists: A list of rank lists. Each rank list should be a list of (item, rank) tuples.
    :param alpha: The parameter alpha used in the RRF formula. Default is 60.
    :param default_rank: The default rank assigned to items not present in a rank list. Default is 1000.
    :return: Sorted list of items based on their RRF scores.
    """
    # Consolidate all unique items from all rank lists
    all_items = set(item for rank_list in rank_lists for item, _ in rank_list)

    # Create a mapping of items to indices
    item_to_index = {item: idx for idx, item in enumerate(all_items)}

    # Initialize a matrix to hold the ranks, filled with the default rank
    rank_matrix = np.full((len(all_items), len(rank_lists)), default_rank)

    # Fill in the actual ranks from the rank lists
    for list_idx, rank_list in enumerate(rank_lists):
        for item, rank in rank_list:
            rank_matrix[item_to_index[item], list_idx] = rank

    # Calculate RRF scores using NumPy operations
    rrf_scores = np.sum(1.0 / (alpha + rank_matrix), axis=1)

    # Sort items based on RRF scores
    sorted_indices = np.argsort(-rrf_scores)  # Negative for descending order

    # Retrieve sorted items
    sorted_items = [(list(item_to_index.keys())[idx], rrf_scores[idx]) for idx in sorted_indices]

    return sorted_items

def rank_list(search_result: List[ScoredPoint]):
    return [(point.id, rank + 1) for rank, point in enumerate(search_result)]

dense_rank_list, sparse_rank_list = rank_list(search_results[0]), rank_list(search_results[1])

rrf_rank_list = rrf([dense_rank_list, sparse_rank_list])

def find_point_by_id(
    client: QdrantClient, collection_name: str, rrf_rank_list: List[Tuple[int, float]]
):
    return client.retrieve(
        collection_name=collection_name, ids=[item[0] for item in rrf_rank_list]
    )

found_results = find_point_by_id(qdrant_client, collection_name, rrf_rank_list)


In [32]:
print(found_results)

[Record(id=10, payload={'title': 'Conceptbesluitenlijst raadsvergadering 1feb2022.docx', 'location_name': 'Staphorst', 'location': 'GM0180', 'text': 'Titel: Conceptbesluitenlijst raadsvergadering 1feb2022.docx \n Gemeente: Staphorst \n Tekst: \n\n Wethouder Mussche doet de toezegging schriftelijke terug te komen op de vraag welke afspraken gemaakt zijn met de hengelaarsvereniging voor wat betreft de bermen. Eveneens doet wethouder Mussche de toezegging na te gaan of er vragen liggen van de agrarirs over de invulling van de bermen. 10. Opinirend: Rekenkameronderzoek afvalbeleid Staphorst Na beraadslaging in twee termijnen concludeert de voorzitter dat dit onderwerp als besluitvormend hamerstuk wordt geagendeerd voor de raadsvergadering', 'source': 'openbesluitvorming', 'es_id': '651b1e224f177a6bf2bca6916f7fda7368762716', 'modified': '2022-02-15T19:30:00+00:00', 'published': '2022-02-15T19:30:00+00:00', 'type': 'Bestand', 'identifier': 'https://id.openraadsinformatie.nl/4318661', 'url': 